In [1]:
from typing import Optional,Tuple
import torch
import torch.nn as nn

In [2]:
# This file implements a Vision Transformer (ViT) model that processes images by:
# 1. Splitting the image into patches
# 2. Converting each patch into an embedding
# 3. Adding positional embeddings
# 4. Processing through transformer layers
# 5. Outputting final image features

In [14]:
class VisionConfig:
    
    def __init__(
        self,
        hidden_size=768,  # Size of the embeddings used throughout the model
        intermediate_size=3072,  # Size of the intermediate layer in MLP
        num_hidden_layers=12,  # Number of transformer layers
        num_attention_heads=12,  # Number of attention heads in each transformer layer
        num_channels=3,  # Number of input image channels (3 for RGB)
        image_size=224,  # Input image size (224x224 pixels)
        patch_size=16,  # Size of each image patch (16x16 pixels)
        layer_norm_eps=1e-6,  # Small constant for numerical stability in layer norm
        attention_dropout=0.0,  # Dropout rate for attention
        num_image_tokens: int = None,  # Number of image tokens (patches) - calculated as (image_size/patch_size)^2
        **kwargs
    ):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_channels = num_channels
        self.patch_size = patch_size
        self.image_size = image_size
        self.attention_dropout = attention_dropout
        self.layer_norm_eps = layer_norm_eps
        self.num_image_tokens = num_image_tokens
        

In [4]:
"""Converts input images into patch embeddings and adds positional embeddings"""

'Converts input images into patch embeddings and adds positional embeddings'

In [15]:
class VisionEmbeddings(nn.Module):
    
    def __init__(self,config: VisionConfig):
        
        super().__init__()
        self.config = config
        self.embed_dim = config.hidden_size
        self.image_size = config.image_size
        self.patch_size = config.patch_size
        
        self.patch_embedding = nn.Conv2d(
            in_channels=config.num_channels,
            out_channels=self.embed_dim,
            kernel_size=self.patch_size,
            stride=self.patch_size,
            padding='valid'
        )
        
        self.num_patches = (self.image_size // self.patch_size) ** 2
        self.num_positions = self.num_patches
        
        self.position_embedding = nn.Embedding(self.num_positions,self.embed_dim)
        
        self.register_buffer(
            "position_ids",
            torch.arange(self.num_positions).expand((1,-1)),
            persistent=False
        )
        
    def forward(self,pixel_values: torch.FloatTensor):
        
        _,_,height,width = pixel_values.shape
        
        patch_embeds = self.patch_embedding(pixel_values)
        print('patch1 shape',patch_embeds.shape)
        
        embeddings = patch_embeds.flatten(2)
        print('emb shape',embeddings.shape)
        
        embeddings = embeddings.transpose(1,2)
        print('emb1 shape',embeddings.shape)
        
        embeddings = embeddings + self.position_embedding(self.position_ids)
        print('embeddings shape',embeddings.shape)

        return embeddings

In [16]:
config = VisionConfig(hidden_size=128, image_size=256, patch_size=16, num_channels=3)

In [17]:
# Initialize the VisionEmbeddings model
model = VisionEmbeddings(config)

# Create a sample input tensor (batch_size=1, channels=3, height=256, width=256)
input_tensor = torch.randn(1, 3, 256, 256)

# Pass the input tensor through the model
output = model(input_tensor)

print("Output shape:", output.shape)  # Should be (1, num_patches, embed_dim)

patch1 shape torch.Size([1, 128, 16, 16])
emb shape torch.Size([1, 128, 256])
emb1 shape torch.Size([1, 256, 128])
embeddings shape torch.Size([1, 256, 128])
Output shape: torch.Size([1, 256, 128])


In [26]:
class Attention(nn.Module):
    
    def __init__(self,config):
        
        super().__init__()
        
        self.config = config
        self.embed_dim = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.embed_dim // self.num_heads
        
        self.scale = self.head_dim ** -0.5
        self.dropout = config.attention_dropout
        
        self.k_proj = nn.Linear(self.embed_dim,self.embed_dim)
        self.v_proj = nn.Linear(self.embed_dim,self.embed_dim)
        self.q_proj = nn.Linear(self.embed_dim,self.embed_dim)
        
        self.out_proj = nn.Linear(self.embed_dim,self.embed_dim)
        
    
    def forward(self,hidden_states: torch.Tensor) -> Tuple[torch.Tensor,
                                                          Optional[torch.Tensor]]:
        
        """Apply multi-headed self-attention to the input.
        
        Args:
            hidden_states (torch.Tensor): Input tensor of shape [Batch_Size, Num_Patches, Embed_Dim]
            
        Returns:
            Tuple[torch.Tensor, Optional[torch.Tensor]]: 
                - Attention output of shape [Batch_Size, Num_Patches, Embed_Dim]
                - Attention weights of shape [Batch_Size, Num_Heads, Num_Patches, Num_Patches]
        """
        batch_size, seq_len, _ = hidden_states.size()
        
        query_states = self.q_proj(hidden_states)
        
        key_states = self.k_proj(hidden_states)
        
        value_states = self.v_proj(hidden_states)
        
        query_states = query_states.view(batch_size, seq_len,self.num_heads,self.head_dim).transpose(1,2)
        
        key_states = key_states.view(batch_size, seq_len,self.num_heads,self.head_dim).transpose(1,2)

        value_states = value_states.view(batch_size, seq_len,self.num_heads,self.head_dim).transpose(1,2)
        
        
        attn_weights = (torch.matmul(query_states,key_states.transpose(2,3))*self.scale )
        
        if attn_weights.size() != (batch_size, self.num_heads, seq_len, seq_len):
            raise ValueError(
                f"Attention weights should be of size {(batch_size, self.num_heads, seq_len, seq_len)}, but is"
                f" {attn_weights.size()}"
            )
        
        attn_weights = nn.functional.softmax(attn_weights,dim=-1,dtype=torch.float32).to(query_states.dtype)
        
        attn_weights = nn.functional.dropout(attn_weights,p=self.dropout,training=self.training)
        
        attn_output = torch.matmul(attn_weights,value_states)
        
        
        if attn_output.size() != (batch_size, self.num_heads, seq_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(batch_size, self.num_heads, seq_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )
        
        attn_output = attn_output.transpose(1,2).contiguous()
        
        attn_output = attn_output.reshape(batch_size,seq_len,self.embed_dim)

        attn_output = self.out_proj(attn_output)
        
        return attn_output,attn_weights

In [27]:
config = VisionConfig(hidden_size = 128,num_attention_heads = 8,attention_dropout = 0.1)
attention_layer = Attention(config)
# 假设输入的 hidden_states 形状为 [batch_size, seq_len, embed_dim]
hidden_states = torch.randn(32, 10, 128)  # batch_size=32, seq_len=10, embed_dim=128

# 前向传播
attn_output, attn_weights = attention_layer(hidden_states)

# 输出的形状
print(attn_output.shape)  # 应该是 [batch_size, seq_len, embed_dim]
print(attn_weights.shape)  # 应该是 [batch_size, num_heads, seq_len, seq_len]

torch.Size([32, 10, 128])
torch.Size([32, 8, 10, 10])
